In [14]:
#!/usr/bin/env python3
import tensorflow as tf
import json

import os

from src.ForecastModel.data.models import DataModelCV
from src.ForecastModel.utils.metrics import calculate_nse, calculate_kge, calculate_bias, calculate_rms, calculate_bias_flv, calculate_bias_fhv
from src.ForecastModel.utils.losses import loss_peak_mse
from src.ForecastModel.utils.postprocessing import ModelHandler, df2latex, get_bold_mask, load_metrics, get_n_peaks, dt

import numpy as np
import pandas as pd

In [2]:
PLOT_PATH          = r"plots"
DATA_PATH          = r"src\data\Dataset.csv"
CROSS_INDICES_PATH = r"src\data\indices"

In [3]:
models = {
    "arima": ModelHandler("ARIMA",
                r"rst\ARIMA",
                is_final_model = True,
                is_external_model = True,
                color = "#999999",
                ls = "--",
                  ),
     "elstm": ModelHandler("eLSTM",
                   r"rst\eLSTM",
                   is_final_model = True,
                   color = '#984ea3',
                   ls = "-",
                 ),
     "pbhm-hlstm": ModelHandler("PBHM-HLSTM",
               r"rst\PBHM-HLSTM",
               is_final_model = True,
               color = "#e41a1c",
               ls = "-",
              )
     }

# Table 1: Statistics of the catchment

In [4]:
# calculate metrics of hydrologic model per fold

dfh = pd.read_csv(DATA_PATH, parse_dates=["time"])

hyd_metrics = {}
for year in range(2011, 2018):
      dfp = dfh.loc[dfh["time"].dt.year == year]
      hyd_metrics[year] = {
          "kge":  calculate_kge(dfp.qmeasval.values, dfp.qsim.values),
          "nse":  calculate_nse(dfp.qmeasval.values, dfp.qsim.values),
          "bias": calculate_bias(dfp.qmeasval.values, dfp.qsim.values),
          "q_mean": dfp.qmeasval.mean(),
          "q_std":  dfp.qmeasval.std(),
          "q_max":  dfp.qmeasval.max(),
          "q_sum":  dfp.qmeasval.sum()*0.25*60*60 / 1e6,
          "qs_mean": dfp.qsim.mean(),
          "qs_std":  dfp.qsim.std(),
          "qs_max":  dfp.qsim.max(),
          "qs_sum":  dfp.qsim.sum()*0.25*60*60 / 1e6,
          "pmx_max": dfp.pmax.max()*4,
          "pmx_sum": dfp.pmax.sum(),
          "p_max":   dfp.pmean.max()*4,
          "p_sum":   dfp.pmean.sum(),
          "t_mean": dfp.tmean.mean(),
          "t_std":  dfp.tmean.std(),
      }    


In [5]:
df_catchment_stats = pd.DataFrame(hyd_metrics).transpose()
df_catchment_stats = df_catchment_stats[df_catchment_stats.columns[3:]].transpose()
df_catchment_stats.columns = [str(x) for x in df_catchment_stats.columns]
df_catchment_stats

,2011,2012,2013,2014,2015,2016,2017
q_mean,0.571149,1.009310,1.208354,1.167666,0.711204,0.829312,0.567430
q_std,0.245171,0.842681,0.681078,0.669319,0.332219,0.682278,0.229892
q_max,9.610000,25.200000,15.000000,7.270000,5.850000,17.940000,9.210000
q_sum,18.011757,31.916797,38.106657,36.823518,22.428531,26.224821,17.698878
qs_mean,0.617823,1.103376,1.399735,1.349319,0.755306,0.916051,0.907808
qs_std,0.331923,0.832882,1.016826,0.719722,0.519150,0.701360,0.479473
qs_max,4.195333,8.937861,11.354018,7.681351,4.502031,6.426838,4.611206
qs_sum,19.483660,34.891398,44.142029,42.552124,23.819337,28.967737,28.315708
pmx_max,118.720001,180.520004,100.480003,84.639999,109.919998,231.880005,173.360001
pmx_sum,2159.559998,2877.670002,2777.400000,2847.860000,2198.090000,3222.800001,3076.049996


In [6]:
df2latex(df_catchment_stats, os.path.join(PLOT_PATH, r"table_1_summary_data.txt"))

plots\table_1_summary_data.txt


# Table 2: Average Model Performance

In [29]:
metric_names = ["kge", "nse", "bias", "flv", "fhv"]
metric_labels = ["KGE", "NSE", "PBIAS", "FLV", "FHV"]

xx = np.arange(1,97)
df = pd.DataFrame(columns=["name", "year"])

n_row = -5

for n, key in enumerate(models.keys()):
    metrics          = load_metrics(os.path.join(models[key].lg_path, "metrics_eval.txt"))
    metrics_baseline = load_metrics(os.path.join(models["arima"].lg_path, "metrics_eval.txt"))
    
    n_row += 5
    
    for j, met in enumerate(metric_names):

        metric_test     = metrics["test"][met]
        metric_baseline = metrics_baseline["test"][met]
        
        for i in range(5):
            df.loc[n_row+i, ["name", "year"]] = [models[key].name, 2013+i]

            # evaluations --------------------------------------------------------------------------
            # median metrics of test set
            df.loc[n_row+i, [f"{met}_test"]]   = [np.median(metric_test[i])]
            df.loc[n_row+i, [f"{met}_min"]]   = [np.min(metric_test[i])]
            df.loc[n_row+i, [f"{met}_max"]]   = [np.max(metric_test[i])]
            # metric for inital and final step in forecasting window
            df.loc[n_row+i, [f"{met}_init"]]   = [metric_test[i][0]]
            df.loc[n_row+i, [f"{met}_final"]]  = [ metric_test[i][-1]]
            # difference of metric between inital and final step in forecasting window
            df.loc[n_row+i, [f"{met}_drop"]]   = [metric_test[i][0] - metric_test[i][-1]]
            # num of lead time steps required so that model outperforms the baseline (ARIMA)
            if met != "bias":
                out_perform_idxs = [n for n, (x,y) in enumerate(zip(metric_baseline[i], metric_test[i])) if y-x > 0]
                if len(out_perform_idxs) > 0:
                    df.loc[n_row+i, [f"{met}_out"]] = out_perform_idxs[0]
                    df.loc[n_row+i, [f"{met}_out_hours"]] = out_perform_idxs[0] * 0.25
                else:
                    df.loc[n_row+i, [f"{met}_out"]] = -1
                    df.loc[n_row+i, [f"{met}_out_hours"]] = -1

# summary
cols = df.columns.to_list()
df.filter(like="kge")
#df[cols[:2]+cols[4:9]+cols[12:17]+cols[20:]]

,kge_test,kge_min,kge_max,kge_init,kge_final,kge_drop,kge_out,kge_out_hours
0,0.885991,0.883965,0.992364,0.992364,0.883965,0.108399,-1.0,-1.00
1,0.869743,0.865033,0.995619,0.995619,0.869098,0.126521,-1.0,-1.00
2,0.800245,0.726017,0.995044,0.995044,0.726017,0.269026,-1.0,-1.00
3,0.828989,0.806670,0.992716,0.992716,0.806670,0.186046,-1.0,-1.00
4,0.580995,0.536915,0.968628,0.968628,0.594178,0.374450,-1.0,-1.00
5,0.891144,0.849049,0.947633,0.947633,0.897098,0.050535,9.0,2.25
6,0.858158,0.787249,0.900123,0.900123,0.787249,0.112874,43.0,10.75
7,0.887542,0.704111,0.955778,0.885573,0.704111,0.181462,13.0,3.25
8,0.668890,0.614212,0.851625,0.851625,0.614212,0.237413,-1.0,-1.00
9,0.798162,0.647341,0.850612,0.850612,0.647341,0.203271,5.0,1.25


In [8]:
# calculate metrics of hydrologic model per fold
dfh = pd.read_csv(DATA_PATH, parse_dates=["time"])

hyd_metrics = {}
for year in range(2012, 2018):
      dfp = dfh.loc[dfh["time"].dt.year == year]
      hyd_metrics[year] = {
          "kge":  calculate_kge(dfp.qmeasval.values, dfp.qsim.values),
          "nse":  calculate_nse(dfp.qmeasval.values, dfp.qsim.values),
          "bias": calculate_bias(dfp.qmeasval.values, dfp.qsim.values),
          "flv": calculate_bias_flv(dfp.qmeasval.values, dfp.qsim.values),
          "fhv": calculate_bias_fhv(dfp.qmeasval.values, dfp.qsim.values),
      }    
    
    
# averaged metrics
evalu = {}
for n, key in enumerate(models.keys()):
    metrics = load_metrics(os.path.join(models[key].lg_path, "metrics_eval.txt"))
    evalu[key] = {}
    for i in range(0,5):
        year = 2013 + i
        evalu[key][year] = {}
        for j, met in enumerate(["kge", "nse", "bias", "flv", "fhv"]):
            metric = metrics["test"][met][i]
            evalu[key][year][met] = metric
       

In [9]:
years = np.array([x for x in range(2013,2018)])
metric_names = ["kge", "nse", "bias", "flv", "fhv"]

df_avg_model = pd.DataFrame(index=years)

# get metrics from the hydrological model
for year in years:
    for met in metric_names:
        df_avg_model.loc[year, f"hyd_{met}"] = hyd_metrics[year][met]

# get metrics from data models
for n,key in enumerate(evalu.keys()):
    for year in years:
        for met in metric_names:
            df_avg_model.loc[year, f"{key}_{met}"] = np.median(evalu[key][year][met])

df_avg_model

,hyd_kge,hyd_nse,hyd_bias,hyd_flv,hyd_fhv,arima_kge,arima_nse,arima_bias,arima_flv,arima_fhv,elstm_kge,elstm_nse,elstm_bias,elstm_flv,elstm_fhv,pbhm-hlstm_kge,pbhm-hlstm_nse,pbhm-hlstm_bias,pbhm-hlstm_flv,pbhm-hlstm_fhv
2013,0.631525,0.185013,15.838105,29.212907,32.441052,0.885991,0.769669,0.419290,48.702402,-0.775360,0.891144,0.841174,-0.918990,8.924938,2.679881,0.873886,0.915637,-4.399931,-129.741420,1.986731
2014,0.738640,0.494420,15.556921,-130.621654,6.862243,0.869743,0.763630,0.866919,6.087115,6.403875,0.858158,0.905239,2.037583,-141.644974,-12.417576,0.939515,0.949100,-3.055125,-136.649814,-8.349699
2015,0.505049,0.236257,6.201059,16.722689,35.474510,0.800245,0.670728,0.710480,43.057975,24.910782,0.887542,0.905640,1.221196,38.667841,-5.346377,0.869994,0.878783,7.679015,-18.462809,4.522243
2016,0.738958,0.512468,10.459236,-46.968768,-5.176013,0.828989,0.683637,0.303081,18.348574,-6.162203,0.668890,0.671255,-8.790850,-119.893203,-39.579754,0.892467,0.876417,-2.539703,-1.471710,-10.270599
2017,0.191555,-4.243535,59.985891,-151.860726,74.533865,0.580995,0.232782,0.559718,10.826032,7.066954,0.798162,0.656459,-0.235100,-194.327971,0.945907,0.830826,0.697210,12.175733,-306.760369,11.077637


In [10]:
columns = [x for x in df_avg_model.columns if x.split("_")[1] in ["kge", "nse", "bias", "fhv"]]
mask = get_bold_mask(df_avg_model[columns].abs(), [np.argmax, np.argmax, np.argmin, np.argmin], 4, 0)
df2latex(df_avg_model[columns], 
         os.path.join(PLOT_PATH, r"table_2_mean_metrics.txt"), 
         mask, 
         ['6.2f', '6.2f', '+6.1f', '+6.1f']*4, 
        )

columns = [x for x in df_avg_model.columns if x.split("_")[1] in ["bias", "flv", "fhv"]]
mask = get_bold_mask(df_avg_model[columns].abs(), [np.argmin, np.argmin, np.argmin], 3, 2)
df2latex(df_avg_model[columns], os.path.join(PLOT_PATH, r"table_3_mean_metrics-bias.txt"), mask, '6.1f')

plots\table_2_mean_metrics.txt
plots\table_3_mean_metrics-bias.txt


# Table 4: Peak Performance

In [11]:
def dt(dates, format="%d/%m/%Y %H:%M"):
    if dates.tz == None:
        # make TZ aware
        return pd.to_datetime(dates, format=format).tz_localize("Europe/London").tz_convert("UTC")
    else:
        return pd.to_datetime(dates, format=format).tz_convert("UTC")

In [12]:
# def
num_peaks_per_fold = 2    # number of peaks per fold to analyze
load_predictions   = False # load predictions or newly predict with models

In [16]:
idx = -10
dfp = pd.DataFrame(columns = ["name", "year", "peak", 
                              "peak_flow", "total_flow",
                              "hyd_perr", "hyd_poff",
                              "rms_hyd", "flow_hyd", 
                              "rms_0", "rms_m", "rms_95",
                              "flow_0", "flow_m", "flow_95"]
                   
                        ) 
dfp = dfp.astype(dtype= {"name"     :"str",     "year"      :"int32",    "peak"  : "int32", 
                         "peak_flow":"float64", "total_flow":"float64",
                         "hyd_perr" :"float64", "hyd_poff"  :"float64",
                         "rms_hyd"  :"float64", "flow_hyd"  :"float64", 
                         "rms_0"    :"float64", "rms_m"     :"float64", "rms_95" : "float64",
                         "flow_0"   :"float64", "flow_m"    :"float64", "flow_95": "float64"}
                )

for n, key in enumerate(models.keys()):
    idx += 10
    print(key)
    eval_path = os.path.join(models[key].hp_path, "eval_peaks.pkl")
    
    if not os.path.exists(eval_path) or load_predictions == False:
        eval_peaks = []

        # load datamodel
        dm = DataModelCV(DATA_PATH,
               target_name       = models[key].target_name,
               hincast_features  = models[key].feat_hindcast,
               forecast_features = models[key].feat_forecast,
             )

        if models[key].is_external_model:
            overlap_length = 0
            hindcast_length = 96
        else:
            # load trial data
            with open(os.path.join(models[key].hp_path, "trial.json")) as f:
                trial = json.load(f)

            hindcast_length = trial['hyperparameters']['values']['hindcast_length']
            try:
                overlap_length = trial['hyperparameters']['values']['osc_length']
            except:
                overlap_length = 0 

        dm.main(os.path.join(CROSS_INDICES_PATH, f"cross_indices_{hindcast_length}.pkl"))

        for n_fold in dm.cross_sets.keys():
            year = 2013 + n_fold

            # load dataset
            X, y  = dm.getDataSet(dm.cross_sets[n_fold]["test"], scale=True) 

            # get hydrologial model 
            s = dm.getFeatureSet(n_fold+2, "qsim")[2]
            df = pd.DataFrame({'index':dt(s.index), 'qhyd':s.values}).set_index("index")

            # add ground truth
            s = dm.getFeatureSet(n_fold+2, "qmeasval")[2]
            s.index = dt(s.index)
            df = df.merge(s.rename("qmeas").to_frame(), left_index=True, right_index=True)

            if models[key].is_external_model:
                
                ext_df = pd.read_pickle(os.path.join(models[key].hp_path, f"forecast_{year}.pkl"))

                ext_df.index = pd.date_range(ext_df.index[0], ext_df.index[-1], freq="15min", tz="UTC")
                
                forecasts_df = ext_df[[f"fc{x:d}" for x in range(96)]].copy()
                forecasts_df.columns = [f"q{x:d}" for x in range(96)]

                del ext_df
                
            else:
                # load model
                tf.keras.backend.clear_session()
                model  = tf.keras.models.load_model(os.path.join(models[key].hp_path, f"model_fold_{n_fold:d}.keras"))

                yp = model.predict(X, batch_size=1000)
                
                if key == "lstm_residual":
                    _, _, yidx = dm.sets[dm.cross_sets[n_fold]["test"]]
                    simu = dm.getWithIndexArray(["qsim"], yidx)
            
                    # get real values from residuals
                    yp += simu[:,:,0]
            
                forecasts_df = pd.DataFrame(data    = yp, 
                                        columns = [f"q{x:d}" for x in range(yp.shape[1])],
                                        index   = dt(dm.getTimeSet(n_fold+2, 0)[2]))

                # save dataframe 
                df_out = forecasts_df.copy()
                df_out.index = pd.to_datetime(df_out.index, format="%d/%m/%Y %H:%M", utc=True)
                df_out.to_pickle(os.path.join(models[key].hp_path, f"forecast_{year}.pkl"))
            
            
            # get forcasting stats                          
            for forecast_step in range(1, forecasts_df.shape[1]):
                forecasts_df[f"q{forecast_step:d}"] = forecasts_df[f"q{forecast_step:d}"].shift(forecast_step)
            
            # merge model predctions
            df = df.merge(forecasts_df, left_index=True, right_index=True)  
            
            # merge prcipitation
            s = pd.Series(dm.getFeatureSet(n_fold+2, "pmean", 0)[2].values, dt(dm.getTimeSet(n_fold+2, 0)[2]))
            df = df.merge(s.rename("pmean").to_frame(), left_index=True, right_index=True)
            
            forecasts_df.dropna(inplace=True)
            stats_df = pd.DataFrame(columns = ["fmin", "fmax", "fmean", 
                                               "fq95", "fq90", "fq75",
                                               "fq50",
                                               "fq25", "fq10", "fq5"],
                                   index = forecasts_df.index)
            
            for i, row in forecasts_df.iterrows():
                stats_df.loc[i] = [row.values.min(), row.values.max(), row.values.mean()] + \
                                        [np.quantile(row.values, float(x[2:])/100) for x in stats_df.columns[3:]]
                  
            # merge stats
            df = df.merge(stats_df, left_index=True, right_index=True)        
            df.dropna(inplace=True)
                                           
            peaks = get_n_peaks(df, "qmeas", num_peaks_per_fold, 24*4)
            peaks["n_fold"] = n_fold + 2

            # add to summary
            eval_peaks.append(peaks)

        # save data to pickle
        df = pd.concat(eval_peaks, axis=0)
        df.to_pickle(eval_path)

    df = pd.read_pickle(eval_path)
    for n_fold in df.n_fold.unique().tolist():
        print(f"processing fold {n_fold}")
        peaks = df[df.n_fold == n_fold]
        for p in range(num_peaks_per_fold):

            # eval peaks
            idx_peak  = peaks[peaks.n_peak == p]["qmeas"].argmax()
            peak_flow = peaks[peaks.n_peak == p]["qmeas"].max()
            dfp.loc[idx+n_fold+5*p, ["hyd_perr", "hyd_poff"]] = [peaks[peaks.n_peak == p]["qhyd"].max() - peak_flow,
                                                                 peaks[peaks.n_peak == p]["qhyd"].argmax() - idx_peak,
                                                                ]
            dfp.loc[idx+n_fold+5*p, [f"perr_{x}" for x in range(96)]] = [peaks[peaks.n_peak == p][f"q{x}"].max() - peak_flow for x in range(96)]
            dfp.loc[idx+n_fold+5*p, [f"poff_{x}" for x in range(96)]] = [peaks[peaks.n_peak == p][f"q{x}"].argmax() - idx_peak for x in range(96)]

            # eval section
            rms_q0 = calculate_rms(peaks[peaks.n_peak == p]["qmeas"].values, 
                                    peaks[peaks.n_peak == p]["q0"].values)
            rms_qm = calculate_rms(peaks[peaks.n_peak == p]["qmeas"].values, 
                                    peaks[peaks.n_peak == p]["fmean"].values)
            rms_q95 = calculate_rms(peaks[peaks.n_peak == p]["qmeas"].values, 
                                    peaks[peaks.n_peak == p]["q95"].values)
            rms_hyd = calculate_rms(peaks[peaks.n_peak == p]["qmeas"].values, 
                                    peaks[peaks.n_peak == p]["qhyd"].values)
            
            peak_flow = peaks[peaks.n_peak == p]["qmeas"].max()
            
            dfp.loc[idx+n_fold+5*p, ["name", "year", "peak"]] = [models[key].name, np.int32(2011+n_fold), np.int32(p)]
            dfp.loc[idx+n_fold+5*p, ["peak_flow", "total_flow"]] = [peak_flow, peaks[peaks.n_peak == p]["qmeas"].sum()]
            dfp.loc[idx+n_fold+5*p, ["rms_hyd", "flow_hyd"]] = [rms_hyd,  peaks[peaks.n_peak == p]["qhyd"].sum()]
            dfp.loc[idx+n_fold+5*p, ["rms_0", "rms_m", "rms_95"]] = [rms_q0, rms_qm, rms_q95]
            dfp.loc[idx+n_fold+5*p, ["flow_0", "flow_m", "flow_95"]] = [peaks[peaks.n_peak == p]["q0"].sum(),
                                                                     peaks[peaks.n_peak == p]["fmean"].sum(),
                                                                     peaks[peaks.n_peak == p]["q95"].sum(),
                                                                    ]

arima
dictonary loaded
processing fold 2
processing fold 3
processing fold 4
processing fold 5
processing fold 6
elstm
dictonary loaded
35/35 [==============================] - 1s 14ms/step
processing fold 2
processing fold 3
processing fold 4
processing fold 5
processing fold 6
pbhm-hlstm
dictonary loaded
35/35 [==============================] - 1s 14ms/step
processing fold 2
processing fold 3
processing fold 4
processing fold 5
processing fold 6


In [62]:
dfout = dfp[["name", "year", "peak", "peak_flow", "hyd_perr", "hyd_poff"]].copy()
dfout["peak_median"] = dfp.filter(regex='^perr').median(axis=1)
dfout["off_median"]  = dfp.filter(regex='^poff').median(axis=1)

dfout["hyd_perr"] = 100 * dfout["hyd_perr"] /  dfout["peak_flow"]
dfout["peak_median"] = 100 * dfout["peak_median"] /  dfout["peak_flow"]

dfout = dfout.reset_index()
df = dfout.loc[0:9, dfout.columns[2:7]]

for n, key in enumerate(models.keys()):
    df = df.join(dfout.loc[n*10:n*10+10, dfout.columns[7:9]].reset_index(drop=True), how='left', rsuffix="_" + models[key].name)

df.index = pd.MultiIndex.from_arrays([df["year"].map(int), df["peak"].map(int)])
df = df.drop(["peak", "year"], axis=1)

# get summary
for col in df.columns:
    df.loc[("all folds", 0), col] = df[col].iloc[0:10:2].median()
    df.loc[("all folds", 1), col] = df[col].iloc[1:10:2].median()
    df.loc[("all folds", 99),col] = df[col].iloc[0:11].median()

In [ ]:
#ALTERNATIVE REMOVE

# get summary statistics over all events
# hydrologic model
df.loc[("all folds", 0), "hyd_perr"] = dfout["hyd_perr"].iloc[:11:2].median()
df.loc[("all folds", 1), "hyd_perr"] = dfout["hyd_perr"].iloc[1:11:2].median()
df.loc[("all folds", 99), "hyd_perr"] = dfout["hyd_perr"].median()

df.loc[("all folds", 0), "hyd_poff"] = dfout["hyd_poff"].iloc[:11:2].median()
df.loc[("all folds", 1), "hyd_poff"] = dfout["hyd_poff"].iloc[1:11:2].median()
df.loc[("all folds", 99), "hyd_poff"] = dfout["hyd_poff"].median()

# models percentage error
dfall = dfp.filter(regex='^perr').reset_index().drop("index", axis=1)
dfall = dfall.multiply(100/dfout["peak_flow"].values, axis=0)
for n, key in enumerate(models.keys()):
    if n == 0: # fix as suffix is not applied on first appearance of columns
        name = ""
    else:
        name = "_" + models[key].name    
    df.loc[("all folds", 0), "peak_median"+name] = np.median(dfall.loc[n*10:(n+1)*10-1:2, :].values)
    df.loc[("all folds", 1), "peak_median"+name] = np.median(dfall.loc[n*10+1:(n+1)*10:2, :].values)
    df.loc[("all folds", 99),"peak_median"+name] = np.median(dfall.loc[n*10:(n+1)*10-1, :].values)
    
# models offset error
dfall = dfp.filter(regex='^poff').reset_index().drop("index", axis=1)
for n, key in enumerate(models.keys()):
    if n == 0: # fix as suffix is not applied on first appearance of columns
        name = ""
    else:
        name = "_" + models[key].name    
    df.loc[("all folds", 0), "off_median"+name] = np.median(dfall.loc[n*10:(n+1)*10-1:2, :].values)
    df.loc[("all folds", 1), "off_median"+name] = np.median(dfall.loc[n*10+1:(n+1)*10:2, :].values)
    df.loc[("all folds", 99),"off_median"+name] = np.median(dfall.loc[n*10:(n+1)*10-1, :].values)

df = df.fillna(0)

In [63]:
df = df.reset_index()
df = df.drop(["year"], axis=1)

# make latex table and bold best values per row and metric
mask = get_bold_mask(df.abs(), np.argmin, 2, 2)

# apply table format
df.index = [str(2013+x//2) if x%2==0 else " " for x in df.index]
df.rename(index={'2018':"all folds", '2019':" "}, inplace=True)
df.loc[:, "peak"] = df.loc[:, "peak"].map(str).replace({'0':'1st', '1':'2nd', '99':"both"})


df2latex(df, 
         os.path.join(PLOT_PATH, r"table_4_peak_compare_percent.txt"), 
         mask, 
         ['s', '5.2f', '+5.1f', 'd'] + ['+5.1f', 'd']*len(models.keys()),
        )

plots\table_4_peak_compare_percent.txt


In [64]:
df

,peak,peak_flow,hyd_perr,hyd_poff,peak_median,off_median,peak_median_eLSTM,off_median_eLSTM,peak_median_PBHM-HLSTM,off_median_PBHM-HLSTM
2013,1st,15.00,-90.340227,31.0,-71.737310,47.5,-77.956767,59.5,-38.448216,2.0
,2nd,10.02,13.313555,20.0,-26.128253,19.0,-40.036559,-1.0,-27.455656,-5.0
2014,1st,7.27,3.475723,18.0,4.518715,19.0,-26.956867,0.0,-27.751195,-2.0
,2nd,6.23,23.296167,16.0,19.085950,16.0,-24.817277,2.0,-26.546079,4.0
2015,1st,5.85,-62.460701,36.0,-29.935960,36.0,-66.728347,10.0,-25.197952,2.0
,2nd,3.33,4.708865,49.0,17.401484,44.5,-32.685514,0.0,-13.355610,3.0
2016,1st,17.94,-73.571109,26.0,-45.906519,29.5,-77.066668,5.0,-68.804678,3.0
,2nd,9.99,-45.408064,18.0,-56.734714,20.0,-65.490134,27.5,-43.877083,68.5
2017,1st,9.21,-49.932621,25.0,-48.894701,48.5,-54.471552,3.0,-17.348814,-1.0
,2nd,7.37,-63.082924,28.0,-32.624692,31.0,-59.942489,6.0,8.403218,1.0


# Table 5: Fold Sensitivity - all samples

In [34]:
# load data frame
df = pd.read_pickle(r"rst\ig_all_folds-final_model.pkl")
df.index = df.index + 2013
df.index = df.index.map(str)
df.loc['all folds', df.columns] = df.sum()

# convert to percentage
df = df.div(df.sum(axis=1), axis=0)

mask = get_bold_mask(df.abs(), np.argmax, 1, 0)
df2latex(df,
         os.path.join(PLOT_PATH, r"table_5_sensitivity-all.txt"), 
         mask,
         ['5.2f']*df.shape[1],
        )



plots\table_5_sensitivity-all.txt


In [35]:
df

,h_qsim,h_pmax,h_tmean,h_pmean,h_qmeasval,f_qsim,f_pmax,f_tmean,f_pmean
2013,0.085774,0.004285,0.032212,0.014101,0.545216,0.191874,0.009593,0.078738,0.038207
2014,0.077653,0.005334,0.039230,0.015868,0.546356,0.181448,0.011311,0.082652,0.040147
2015,0.076531,0.004859,0.075624,0.013184,0.505371,0.159807,0.009245,0.123223,0.032156
2016,0.074456,0.007071,0.069750,0.017198,0.490915,0.178545,0.013740,0.104189,0.044135
2017,0.059590,0.005025,0.116647,0.012987,0.323302,0.237078,0.021584,0.156075,0.067713
all folds,0.077262,0.005227,0.056465,0.014875,0.505001,0.187377,0.012119,0.099354,0.042320


# Table 6: Fold Sensitivity - 2 major peaks

In [36]:
# load data frame
df = pd.read_pickle(r"rst\ig_peaks-final-model.pkl")

sum_all = df.sum()
df.loc[('sum', 0), df.columns] = df.xs(0,level=1).sum()
df.loc[('sum', 1), df.columns] = df.xs(1,level=1).sum()
df.loc[('sum', 99), df.columns]= sum_all

# convert to percentage
df = df.div(df.sum(axis=1), axis=0)

# apply table format
df = df.reset_index()
df = df.drop(["level_0"], axis=1)

mask = get_bold_mask(df.abs(), np.argmax, 1, 1)

df.index = [str(2013+x//2) if x%2==0 else " " for x in df.index]
df.rename(index={'2018':"all folds", '2019':" "}, inplace=True)
df.loc[:, "level_1"] = df.loc[:, "level_1"].map(str).replace({'0':'1st', '1':'2nd', '99':"both"})


df2latex(df,
         os.path.join(PLOT_PATH, r"table_6_sensitivity-peaks.txt"), 
         mask,
         ['s'] + ['5.2f']*df.shape[1],
        )

plots\table_6_sensitivity-peaks.txt


In [37]:
df

,level_1,h_qsim,h_pmax,h_tmean,h_pmean,h_qmeasval,f_qsim,f_pmax,f_tmean,f_pmean
2013,1st,0.016237,0.002824,0.015778,0.014614,0.089763,0.039678,0.113098,0.113166,0.594842
,2nd,0.124664,0.007715,0.016141,0.042891,0.199251,0.283485,0.016003,0.118792,0.191056
2014,1st,0.062250,0.003099,0.021453,0.018864,0.323244,0.217965,0.019738,0.105085,0.228302
,2nd,0.067771,0.010356,0.016193,0.055890,0.198045,0.237985,0.055833,0.148492,0.209435
2015,1st,0.012457,0.009663,0.025036,0.027574,0.052334,0.047929,0.122144,0.180153,0.522711
,2nd,0.017173,0.003559,0.006829,0.007140,0.113027,0.158891,0.106906,0.218355,0.368120
2016,1st,0.020133,0.009805,0.014467,0.014652,0.143622,0.103901,0.183668,0.145866,0.363886
,2nd,0.116734,0.007199,0.013655,0.026910,0.209351,0.243814,0.018794,0.099739,0.263805
2017,1st,0.009974,0.001640,0.007301,0.006204,0.024990,0.074084,0.272850,0.158683,0.444274
,2nd,0.003683,0.003553,0.009054,0.006372,0.013208,0.028032,0.244793,0.150064,0.541240
